### Figures for cross-participant regression analysis 
1) Results Figure 3 (average timeseries for within and cross-model with difference, example timeseries)
2) Supplementary Figure S2 (timeseries for all dimensions)


In [3]:
"""
Author: linateichmann
Email: lina.teichmann@nih.gov

    Created on 2023-03-30 12:39:32
    Modified on 2023-03-30 12:39:32
"""

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import mne, os
%matplotlib qt

bids_dir = '/Volumes/THINGS-MEG/THINGS-MEG-bids'
res_folder = f'{bids_dir}/derivatives/meg_paper/output/regression'
font = 'Arial'
text_size = 12
text_size_big = 16
text_size_small = 8

plt.rcParams['font.size'] = text_size
plt.rcParams['font.family'] = font

colours = pd.read_csv(f'{bids_dir}/sourcedata/meg_paper/colors66.txt',header=None)
labels = pd.read_csv(f'{bids_dir}/sourcedata/meg_paper/labels_super_short.txt',header=None)


# load data
def load_epochs(bids_dir,participant,behav):
    epochs = mne.read_epochs(f'{bids_dir}/derivatives/preprocessed/preprocessed_P{participant}-epo.fif',preload=False)
    # THINGS-category number & image number start at 1 (matlab based) so subtracting 1 to use for indexing
    epochs.metadata['things_image_nr'] = epochs.metadata['things_image_nr']-1
    epochs.metadata['things_category_nr'] = epochs.metadata['things_category_nr']-1
    # adding dimensional weights to metadata
    epochs.metadata[['dim'+ str(d+1) for d in range(66)]] = np.nan
    for i in range(len(epochs.metadata)):
        if not np.isnan(epochs.metadata.loc[i,'things_image_nr']):
            epochs.metadata.loc[i,['dim'+ str(d+1) for d in range(66)]] = behav[int(epochs.metadata.loc[i,'things_image_nr']),:]
    return epochs

behav = np.loadtxt(f'{bids_dir}/sourcedata/meg_paper/predictions_66d_elastic_clip-ViT-B-32_visual_THINGS.txt')
epochs = load_epochs(bids_dir,1,behav)

epochs_exp = epochs[(epochs.metadata['trial_type']=='exp')]   
epochs_exp.metadata.reset_index(inplace=True,drop=True)


# load results
all_dat_dims=[]
for p in range(1,5):
    all_dat_dims.append(pd.read_csv(f'{res_folder}/P{p}_linreg_within.csv',index_col=0))
all_dat_dims_cross = pd.read_csv(f'{res_folder}/Linreg_cross_predict-dims.csv',index_col=0)


# change data order based on peak amplitude
avg_dim_data = np.mean(all_dat_dims,axis=0)
sorted_idx = np.flipud(np.argsort(np.max(avg_dim_data,axis=0)))

avg_dim_data_ranked = avg_dim_data[:,sorted_idx]
dim_data_ranked = np.array(all_dat_dims)[:,:,sorted_idx]
colours_ranked = colours.to_numpy()[sorted_idx,:]
labels_ranked = labels.loc[sorted_idx,:]
labels_ranked.reset_index(inplace=True,drop=True)

dim_data_cross_ranked = np.array(all_dat_dims_cross)[:,sorted_idx]

filter_col = [col for col in epochs_exp.metadata.columns if col.startswith('dim')] 
weights_sorted_dims = epochs_exp.metadata.loc[:,filter_col].to_numpy()[:,sorted_idx]



Reading /Volumes/THINGS-MEG/THINGS-MEG-bids/derivatives/preprocessed/preprocessed_P1-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...    1300.00 ms
        0 CTF compensation matrices available
Reading /Volumes/THINGS-MEG/THINGS-MEG-bids/derivatives/preprocessed/preprocessed_P1-epo-1.fif ...
    Found the data of interest:
        t =    -100.00 ...    1300.00 ms
        0 CTF compensation matrices available
Reading /Volumes/THINGS-MEG/THINGS-MEG-bids/derivatives/preprocessed/preprocessed_P1-epo-2.fif ...
    Found the data of interest:
        t =    -100.00 ...    1300.00 ms
        0 CTF compensation matrices available
Reading /Volumes/THINGS-MEG/THINGS-MEG-bids/derivatives/preprocessed/preprocessed_P1-epo-3.fif ...
    Found the data of interest:
        t =    -100.00 ...    1300.00 ms
        0 CTF compensation matrices available
Adding metadata with 18 columns
27048 matching events found
No baseline correction applied
0 projection items activated


In [4]:
# MAKE RESULTS FIGURE
# plt.close('all')
axd = plt.figure(constrained_layout=True,figsize=(8.25, 11.75)).subplot_mosaic(

    """
    .AAA.
    BCDEF
    GHIJK
    .....
    .LLL.
    .....
    .....
    .....
    """)

fig = plt.gcf()


ax = axd['A']
ax.plot(epochs.times*1000, epochs.times*0, 'grey', lw=1, linestyle='--')
ax.plot(epochs.times*1000, np.mean(avg_dim_data_ranked,axis=1),'teal',alpha=0.5,lw=2,label='within-participant')
ax.plot(epochs.times*1000, np.mean(dim_data_cross_ranked,axis=1),'teal',lw=2,label='across-participant')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_xlim([epochs.times[0]*1000,epochs.times[-1]*1000])
ax.set_ylim([-0.01,0.09])
ax.legend(frameon=False,fontsize=text_size_small)
ax.set_ylabel('Correlation', labelpad=10)

ax.set_xticklabels('')

axin2 = ax.inset_axes([-100,-0.07, 1400,0.05], transform= ax.transData)

diff_data = dim_data_cross_ranked-avg_dim_data_ranked
axin2.plot(epochs.times*1000, np.mean(diff_data,axis=1),'k',lw=2)


# get the inflection points and add shaded areas
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches

x = epochs.times*1000
y = np.mean(avg_dim_data_ranked-dim_data_cross_ranked, axis=1)

# Plot windows around points of interest
windowsize = 4
points_of_interest_ms = [125, 200, 400, 600 ]
points_of_interest=[np.where(x==v)[0][0] for v in points_of_interest_ms]

text = [str(i) for i in range(1,len(points_of_interest)+1)]
y_pos = [np.min(y * -1), np.max(y * -1)]
text_y_pos_all = [np.min(y_pos), np.max(y_pos), np.max(y_pos)]

for i, v in enumerate(points_of_interest):
    x_pos = [int(x[v - windowsize // 2]), int(x[v + windowsize // 2])]
    axin2.fill_between(range(x_pos[0], x_pos[1]), y_pos[0], y_pos[1], alpha=0.5, color='gray')

# Create a single legend patch for all shaded areas
legend_patch = mpatches.Rectangle((0, 0), 1, 1, color='gray', alpha=0.5, label='timewindows of interest')
axin2.legend(handles=[legend_patch], loc='lower right', bbox_to_anchor=(1, -0.15), frameon=False, fontsize=8, labelspacing=0.1)

# stylize axis 
axin2.spines['top'].set_visible(False)
axin2.spines['right'].set_visible(False)
axin2.set_title('')
axin2.set_xlim([epochs.times[0]*1000,epochs.times[-1]*1000])

axin2.set_ylabel('Difference')
axin2.set_xlabel('time (ms)')


# add individual timeseries
dim_ranks = [int(i) for i in np.round(np.linspace(0,65,6))]
dim_ranks = np.arange(0,65,7)
dim_ranks = [0,3,5,7,8,9,14,21,28,31]
for rank,ax in zip(dim_ranks,[axd['B'],axd['C'],axd['D'],axd['E'],axd['F'],axd['G'],axd['H'],axd['I'],axd['J'],axd['K']]):
    ax.plot(epochs.times*1000, epochs.times*0, 'grey', lw=1, linestyle='--')

    ax.plot(epochs.times*1000,avg_dim_data_ranked[:,rank],c=colours_ranked[rank,:],lw=2,alpha=0.5,label='within')
    ax.plot(epochs.times*1000,dim_data_cross_ranked[:,rank],c=colours_ranked[rank,:],lw=2,label='across')

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim([-0.05,0.3])

    label_text = f'{labels_ranked.loc[rank,0]} ({sorted_idx[rank]+1})'
    slash_index = label_text.rfind("/")
    bracket_index = label_text.find("(")
    
    # Insert a line break before the first "/"
    if slash_index != -1:  # Check if "/" is found
        label_text = f'{label_text[:slash_index+1]}\n{label_text[slash_index+1:]}'
    else:
        label_text = f'{label_text[:bracket_index]}\n{label_text[bracket_index:]}'
    
    title = ax.set_title(label_text, fontsize=text_size_small, y=0.9)
    ax.set_xlim([epochs.times[0]*1000,epochs.times[-1]*1000])
    ax.legend(frameon=False,fontsize=text_size_small)

    # manually adjust positions
    pos_ax = ax.get_position()
    new_width = 1.1 * pos_ax.width  # Adjust the width factor as needed
    pos_ax.x1 = pos_ax.x0 + new_width
    new_y = pos_ax.y0 - 0.04  # Adjust the y-offset as needed
    pos_ax.y1 = new_y + pos_ax.height
    pos_ax.y0 = new_y
    ax.set_position(pos_ax)

    
for ax in [ axd['G'], axd['H'], axd['I'], axd['J'], axd['K']]:
    pos_ax = ax.get_position()
    new_y = pos_ax.y0 - 0.03  # Adjust the y-offset as needed
    pos_ax.y1 = new_y + pos_ax.height
    pos_ax.y0 = new_y
    ax.set_position(pos_ax)

for ax in [axd['C'],axd['D'],axd['E'],axd['F'],axd['H'],axd['I'],axd['J'],axd['K']]:
    ax.set_yticklabels('')

for ax in [axd['B'],axd['C'],axd['D'],axd['E'],axd['F']]:
    ax.set_xticklabels('')



axd['B'].text(-400,0.3,' ',fontsize=text_size_big*3)
fig.text(0.01,0.97,' ',fontsize=text_size_big*2)

axd['B'].set_ylabel('Correlation')
axd['G'].set_ylabel('Correlation')
axd['I'].set_xlabel('time (ms)')


### C Violinplots
ax = axd['L']
diff_timewindow = []
for i in range(len(points_of_interest)):
    idx = [points_of_interest[i]-windowsize//2,points_of_interest[i]+windowsize//2]
    diff_timewindow.append(np.average(avg_dim_data_ranked[idx[0]:idx[1]]-dim_data_cross_ranked[idx[0]:idx[1]],axis=0))
data = diff_timewindow

# Create a grey violin plot
violin_parts = ax.violinplot(data, showmeans=False, showmedians=True, vert=True,widths=0.7, showextrema=False)

# Customize the violin plot
for pc in violin_parts['bodies']:
    pc.set_facecolor('grey')
    pc.set_edgecolor(None)
    pc.set_alpha(0.4)
violin_parts['cmedians'].set_color('k') 

# Add individual colored points
for i, points in enumerate(data):
    ax.scatter(np.full_like(points, i + 1), points, color='k', s=10, alpha=0.7)
    [ax.plot(i+1,points[ii],'.',color=colours_ranked[ii,:]) for ii in dim_ranks]

# # Connect points with thin dashed lines
for ii in [[0,1],[1,2],[2,3]]:
    for i in dim_ranks:
        ax.plot([ii[0]+1,ii[1]+1], [data[ii[0]][i], data[ii[1]][i]], 'k--', c=colours_ranked[i,:], alpha=0.3)

# Show the plot
ax.set_xticks(range(1,len(points_of_interest)+1), [f'Timewindow\n{int(i)}ms' for i in epochs.times[points_of_interest]*1000])
ax.set_xlabel('')
ax.set_ylabel('Difference correlation\nwithin-across')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# add A and B labels
fig.text(0.01,0.97,'A',fontsize=text_size_big*2)
fig.text(0.01,0.76,'B',fontsize=text_size_big*2)
fig.text(0.01,0.47,'C',fontsize=text_size_big*2)


fig.savefig(f'{bids_dir}/derivatives/meg_paper/figures/Figure3.pdf')

/var/folders/05/n72bx5cn6wg1178kthf1kg5wb87k3k/T/ipykernel_11521/72451530.py:160: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k--" (-> color='k'). The keyword argument will take precedence.
  ax.plot([ii[0]+1,ii[1]+1], [data[ii[0]][i], data[ii[1]][i]], 'k--', c=colours_ranked[i,:], alpha=0.3)


In [12]:
# quick look at what the top dimensions are at each selected timepoint
plt.close('all')
fig,ax = plt.subplots(figsize=(10,30))


for x in range(len(points_of_interest)):
    marker_label = [labels_ranked.loc[rank,0] for rank in np.argsort(diff_timewindow[x])]
    y = diff_timewindow[x][np.argsort(diff_timewindow[x])]

    plt.plot(np.full_like(y,points_of_interest_ms[x]),y,'k.')
    [plt.text(np.full_like(y[i],points_of_interest_ms[x]),y[i],marker_label[i],fontsize=8) for i in range(66)]


ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()


In [10]:
# dynamic plot violinplot
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

plt.close('all')
fig,ax = plt.subplots(num=1)
title = ax.text(0.5,0.85, "", bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},
                transform=ax.transAxes, ha="center")

def init(t=0):
    # Create a grey violin plot
    violin_parts = ax.violinplot(diff_timewindow[t], showmeans=False, showmedians=True, vert=True,widths=0.7, showextrema=False)

    # Customize the violin plot
    for pc in violin_parts['bodies']:
        pc.set_facecolor('grey')
        pc.set_edgecolor(None)
        pc.set_alpha(0.4)
    violin_parts['cmedians'].set_color('k') 

    # Add individual colored points
    ax.scatter(np.full_like(diff_timewindow[t], 1), diff_timewindow[t], color='k', s=10, alpha=0.7)
    [ax.plot(1,diff_timewindow[t][ii],'.',color=colours_ranked[ii,:]) for ii in dim_ranks]

    ax.set_ylim([-0.02,0.15])
    ax.set_ylabel('Difference within-across')

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    fig.tight_layout()
    title.set_text(str(epochs.times[t]*1000) +' ms')
    return fig

def update(t):
    ax.cla()
    violin_parts = ax.violinplot(diff_timewindow[t], showmeans=False, showmedians=True, vert=True,widths=0.7, showextrema=False)

    # Customize the violin plot
    for pc in violin_parts['bodies']:
        pc.set_facecolor('grey')
        pc.set_edgecolor(None)
        pc.set_alpha(0.4)
    violin_parts['cmedians'].set_color('k') 

    # Add individual colored points
    ax.scatter(np.full_like(diff_timewindow[t], 1), diff_timewindow[t], color='k', s=10, alpha=0.7)
    [ax.plot(1,diff_timewindow[t][ii],'.',color=colours_ranked[ii,:]) for ii in dim_ranks]

    ax.set_ylim([-0.02,0.15])
    ax.set_ylabel('Difference within-across')
    ax.set_xticklabels('')
    fig.tight_layout()
    plt.title(str(int(epochs.times[t]*1000)) +' ms')
    return fig

diff_timewindow=[]
for i,v in enumerate(epochs.times):
    diff_timewindow.append(avg_dim_data_ranked[i]-dim_data_cross_ranked[i])

ani = FuncAnimation(fig, update, frames=len(diff_timewindow),init_func=init,blit=False,cache_frame_data=False,repeat=False)
writer = animation.PillowWriter(fps=3)

ani.save(f'{bids_dir}/derivatives/meg_paper/figures/violinplot.gif',writer=writer, dpi=600)


In [11]:
# supplementary figure showing all dimension timecourses
plt.close('all')
fig,axs = plt.subplots(figsize=(8.25, 11.75),num=2,ncols=6,nrows=11,sharex=True,sharey=True)

axs=axs.flatten()

for i,ax in enumerate(axs):
    ax.plot(epochs.times*1000,epochs.times*0,'grey',linestyle='--')
    
    ax.plot(epochs.times*1000,avg_dim_data_ranked[:,i],c=colours_ranked[i,:],alpha=0.3,lw=2)
    ax.plot(epochs.times*1000,dim_data_cross_ranked[:,i],c=colours_ranked[i,:],alpha=1,lw=2)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim([-0.05,0.3])

    ax.set_title(f'{labels_ranked.loc[i,0]}\n({sorted_idx[i]+1})',c='k',fontsize=text_size_small)
    ax.set_xlim([epochs.times[0]*1000,epochs.times[-1]*1000])


fig.supylabel('Correlation: true vs predicted label')
fig.supxlabel('time (ms)')
fig.tight_layout()

fig.savefig(f'{bids_dir}/derivatives/meg_paper/figures/Supplementary_cross_within.pdf')

In [ ]:
#### limit the dims

# MAKE RESULTS FIGURE
# plt.close('all')
axd = plt.figure(constrained_layout=True,figsize=(8.25, 11.75)).subplot_mosaic(

    """
    .AAA.
    BCDEF
    GHIJK
    .....
    .LLL.
    .....
    .....
    .....
    """)

fig = plt.gcf()

avg_dim_data_ranked_ltd = avg_dim_data_ranked[:,0:50]
dim_data_cross_ranked_ltd = dim_data_cross_ranked[:,0:50]


ax = axd['A']
ax.plot(epochs.times*1000, epochs.times*0, 'grey', lw=1, linestyle='--')
ax.plot(epochs.times*1000, np.mean(avg_dim_data_ranked_ltd,axis=1),'b',alpha=0.5,lw=2,label='within-participant')
ax.plot(epochs.times*1000, np.mean(dim_data_cross_ranked_ltd,axis=1),'b',lw=2,label='across-participant')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_xlim([epochs.times[0]*1000,epochs.times[-1]*1000])
ax.set_ylim([-0.01,0.09])
ax.legend(frameon=False,fontsize=text_size_small)
ax.set_ylabel('Correlation', labelpad=10)

ax.set_xticklabels('')

axin2 = ax.inset_axes([-100,-0.07, 1400,0.05], transform= ax.transData)

diff_data = dim_data_cross_ranked_ltd-avg_dim_data_ranked_ltd
axin2.plot(epochs.times*1000, np.mean(diff_data,axis=1),'k',lw=2)


# get the inflection points and add shaded areas
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches

x = epochs.times*1000
y = np.mean(diff_data*-1, axis=1)

# Plot windows around points of interest
windowsize = 4
points_of_interest_ms = [125, 200, 400, 600 ]
points_of_interest=[np.where(x==v)[0][0] for v in points_of_interest_ms]

text = [str(i) for i in range(1,len(points_of_interest)+1)]
y_pos = [np.min(y * -1), np.max(y * -1)]
text_y_pos_all = [np.min(y_pos), np.max(y_pos), np.max(y_pos)]

for i, v in enumerate(points_of_interest):
    x_pos = [int(x[v - windowsize // 2]), int(x[v + windowsize // 2])]
    axin2.fill_between(range(x_pos[0], x_pos[1]), y_pos[0], y_pos[1], alpha=0.5, color='gray')

# Create a single legend patch for all shaded areas
legend_patch = mpatches.Rectangle((0, 0), 1, 1, color='gray', alpha=0.5, label='timewindows of interest')
axin2.legend(handles=[legend_patch], loc='lower right', bbox_to_anchor=(1, -0.15), frameon=False, fontsize=8, labelspacing=0.1)

# stylize axis 
axin2.spines['top'].set_visible(False)
axin2.spines['right'].set_visible(False)
axin2.set_title('')
axin2.set_xlim([epochs.times[0]*1000,epochs.times[-1]*1000])

axin2.set_ylabel('Difference')
axin2.set_xlabel('time (ms)')


# add individual timeseries
dim_ranks = [int(i) for i in np.round(np.linspace(0,65,6))]
dim_ranks = np.arange(0,65,7)
dim_ranks = [0,3,5,7,8,9,14,21,28,31]
for rank,ax in zip(dim_ranks,[axd['B'],axd['C'],axd['D'],axd['E'],axd['F'],axd['G'],axd['H'],axd['I'],axd['J'],axd['K']]):
    ax.plot(epochs.times*1000, epochs.times*0, 'grey', lw=1, linestyle='--')

    ax.plot(epochs.times*1000,avg_dim_data_ranked_ltd[:,rank],c=colours_ranked[rank,:],lw=2,alpha=0.5,label='within')
    ax.plot(epochs.times*1000,dim_data_cross_ranked_ltd[:,rank],c=colours_ranked[rank,:],lw=2,label='across')

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim([-0.05,0.3])

    label_text = f'{labels_ranked.loc[rank,0]} ({sorted_idx[rank]+1})'
    slash_index = label_text.rfind("/")
    bracket_index = label_text.find("(")
    
    # Insert a line break before the first "/"
    if slash_index != -1:  # Check if "/" is found
        label_text = f'{label_text[:slash_index+1]}\n{label_text[slash_index+1:]}'
    else:
        label_text = f'{label_text[:bracket_index]}\n{label_text[bracket_index:]}'
    
    title = ax.set_title(label_text, fontsize=text_size_small, y=0.9)
    ax.set_xlim([epochs.times[0]*1000,epochs.times[-1]*1000])
    ax.legend(frameon=False,fontsize=text_size_small)

    # manually adjust positions
    pos_ax = ax.get_position()
    new_width = 1.1 * pos_ax.width  # Adjust the width factor as needed
    pos_ax.x1 = pos_ax.x0 + new_width
    new_y = pos_ax.y0 - 0.04  # Adjust the y-offset as needed
    pos_ax.y1 = new_y + pos_ax.height
    pos_ax.y0 = new_y
    ax.set_position(pos_ax)

    
for ax in [ axd['G'], axd['H'], axd['I'], axd['J'], axd['K']]:
    pos_ax = ax.get_position()
    new_y = pos_ax.y0 - 0.03  # Adjust the y-offset as needed
    pos_ax.y1 = new_y + pos_ax.height
    pos_ax.y0 = new_y
    ax.set_position(pos_ax)

for ax in [axd['C'],axd['D'],axd['E'],axd['F'],axd['H'],axd['I'],axd['J'],axd['K']]:
    ax.set_yticklabels('')

for ax in [axd['B'],axd['C'],axd['D'],axd['E'],axd['F']]:
    ax.set_xticklabels('')



axd['B'].text(-400,0.3,' ',fontsize=text_size_big*3)
fig.text(0.01,0.97,' ',fontsize=text_size_big*2)

axd['B'].set_ylabel('Correlation')
axd['G'].set_ylabel('Correlation')
axd['I'].set_xlabel('time (ms)')


### C Violinplots
ax = axd['L']
diff_timewindow = []
for i in range(len(points_of_interest)):
    idx = [points_of_interest[i]-windowsize//2,points_of_interest[i]+windowsize//2]
    diff_timewindow.append(np.average(avg_dim_data_ranked_ltd[idx[0]:idx[1]]-dim_data_cross_ranked_ltd[idx[0]:idx[1]],axis=0))
data = diff_timewindow

# Create a grey violin plot
violin_parts = ax.violinplot(data, showmeans=False, showmedians=True, vert=True,widths=0.7, showextrema=False)

# Customize the violin plot
for pc in violin_parts['bodies']:
    pc.set_facecolor('grey')
    pc.set_edgecolor(None)
    pc.set_alpha(0.4)
violin_parts['cmedians'].set_color('k') 

# Add individual colored points
for i, points in enumerate(data):
    ax.scatter(np.full_like(points, i + 1), points, color='k', s=10, alpha=0.7)
    [ax.plot(i+1,points[ii],'.',color=colours_ranked[ii,:]) for ii in dim_ranks]

# # Connect points with thin dashed lines
for ii in [[0,1],[1,2],[2,3]]:
    for i in dim_ranks:
        ax.plot([ii[0]+1,ii[1]+1], [data[ii[0]][i], data[ii[1]][i]], 'k--', c=colours_ranked[i,:], alpha=0.3)

# Show the plot
ax.set_xticks(range(1,len(points_of_interest)+1), [f'Timewindow\n{int(i)}ms' for i in epochs.times[points_of_interest]*1000])
ax.set_xlabel('Difference within-across')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# add A and B labels
fig.text(0.01,0.97,'A',fontsize=text_size_big*2)
fig.text(0.01,0.76,'B',fontsize=text_size_big*2)
fig.text(0.01,0.47,'C',fontsize=text_size_big*2)


fig.savefig(f'{bids_dir}/derivatives/meg_paper/figures/Figure3_ltd.pdf')


